In [ ]:
# Licensed under the Apache License, Version 2.0

# inference.py

from abc import ABC, abstractmethod
from transformers import AutoTokenizer, AutoModelForCausalLM

SUPPORTED_MODELS = ['gemma', 'llama']

class InferencePlatform(ABC):
  """An abstract class for the LLM inference platform we use."""
  @abstractmethod
  def predict(self, prompt: str) -> str:
    pass

class HuggingFace(InferencePlatform):
  """An implementation for using HuggingFace as the platform for LLM inference."""
  def __init__(self):
    self._tokenizer = None
    self._model = None

  def authenticate(self, huggingface_token):
    os.environ['HF_TOKEN'] = huggingface_token

  def setup_model(self, model_name: str):
    if model_name not in SUPPORTED_MODELS:
      raise ValueError(f'Unsupported model: {model_name}')
    if model_name == 'gemma':
      self.tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
      self.model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")
    else:
      self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
      self.model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

  def predict(self, prompt: str) -> str:
    inputs = self.tokenizer.tokenize(prompt)
    generate_ids = self.model.generate(inputs.input_ids)
    return self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True)

class VertexAI(InferencePlatform):
  """An implementation for using Google Cloud's Vertex AI as the platform for
  LLM inference."""
  def __init__(self):
    pass

  def predict(self, prompt: str) -> str:
    pass

In [ ]:
# tree_utils.py

def build_tree(dataset_path):
  """Builds and returns the hierarchical document tree."""
  pass

def search_tree(doc_tree, inference_platform: InferencePlatform):
  """Searches the hierarchical document tree recursively."""
  pass

In [ ]:
# run.py

HF_TOKEN = "" # add your hugging face access token here

def main():
  inference_platform = HuggingFace()
  inference_platform.authenticate(HF_TOKEN)
  inference_platform.setup_model('llama')

  dataset_path = ""
  # Build hierarchical document tree.
  doc_tree = build_tree(dataset_path)

  # Search tree recursively.
  search_tree(doc_tree, inference_platform)

if __name__ == '__main__':
  main()

In [ ]:
# visualize.py

# TODO(james): See if we can reuse the original visualization from the paper.